In [4]:
# TRABAJO FINAL INFORMATICA 1:

# INTEGRANTES:

# OLEA BARBOSA ALMA DE DIOS
# LONDOÑO RESTREPO MANUELA
# CORDERO RAMOS LAUREN VANESSA
# GARCIA SIERRA PAULA CAMILA


import mysql.connector
import json
import módulo
import Módulos_de_tercero

SERVER = 'localhost'
USER = 'informatica1'
PASSWD = 'bio123'
DB = 'info1_20232'

cnx = mysql.connector.connect(
    user=USER, password=PASSWD, host=SERVER, database=DB)

cursor = cnx.cursor()
sql_create = 'CREATE TABLE IF NOT EXISTS info_paciente(Documento INT(11) PRIMARY KEY NOT NULL, Nombre VARCHAR(70), Edad INT(4), Sexo VARCHAR(2), EPS VARCHAR(30), Peso FLOAT, Altura FLOAT, Medida_cintura FLOAT, Medida_cadera FLOAT, Medida_cuello FLOAT)'
cursor.execute(sql_create)
cnx.commit()

usuarios = [{'name': 'Alma Olea', 'user': 'alma.olea', 'passwd': '123'},
            {'name': 'Lauren Cordero', 'user': 'lauren.cordero', 'passwd': '456'},
            {'name': 'Manuela Londoño', 'user': 'manuela.londoño', 'passwd': '789'},
            {'name': 'Angelower Santana', 'user': 'angelower.santana', 'passwd': '248'},
            {'name': 'Paula Garcia', 'user': 'paula.garcia', 'passwd': '197'},
            {'name': '1', 'user': '1', 'passwd': '1'}
            ]

data = open('admin/users.json', mode='w', encoding='utf8')
json.dump(usuarios, data, indent=4, ensure_ascii=False)
data.close()
while True:

    print('--------------------------- SISTEMA DE DIAGNÓSTICO AUTOMÁTICO ---------------------------')

    print('\nInicie sesión con sus credenciales.')
    user = input("\nIngrese su nombre de usuario: ")
    contra = input("Ingrese su contraseña: ")

    correct = False

    for l in usuarios:
        if user == l['user'] and contra == l['passwd']:
            correct = True
            break

    if correct:

        print("\n¡Inicio de sesión exitoso!")

        while True:
            print('\nIngrese una opción del menú:')
            while True:
                menu = input('''
                        Menú:
                        1. Ingreso de información de pacientes
                        2. Buscar paciente
                        3. Administrar usuarios
                        4. Salir
                        > ''')
                output = módulo.validardato(menu)
                if output == True:
                    break
                else:
                    print(output)

            if menu == '1':
                while True:
                    while True:
                        menu1 = input('''
                                    ¿Qué desea hacer?
                                    1. Ingresar un nuevo paciente 
                                    2. Volver al menú inicial
                                    >''')
                        res1 = módulo.validardato(menu1)
                        if res1 == True:
                            break
                        else:
                            print(res1)

                    if menu1 == '1':
                        while True:

                            Id_ = input(
                                'Ingrese el número de documento de identidad del paciente a diagnosticar: ')

                            ver = módulo.ver_paciente(Id_)
                            cursor.execute(ver)
                            ver_ = cursor.fetchall()

                            if ver_ == []:
                                nombreap_ = input(
                                    '\nIngrese el nombre y apellido del paciente a diagnosticar: ')

                                # Validar id con int(Id_)
                                # Validar cuando paula mande la función

                                edad = input('Ingrese la edad del paciente:')
                                Sexo = input(
                                    'Ingrese sexo del paciente, M (Masculino) o F (Femenino): ').upper()
                                Eps = input(
                                    'Ingrese la Eps a la que pertenece el paciente: ')

                                print(
                                    '\nIngrese los siguientes datos biométricos para el diagnóstico del paciente:')
                                peso = input('Peso en kg: ')
                                altura = input('Altura en m:  ')
                                cintura = input(
                                    'Medida de la circunferencia de la cintura en cm: ')
                                cadera = input('Medida de la cadera en cm: ')
                                cuello = input(
                                    'Medida del cuello del paciente en cm: ')
                                print('\n¡Datos biométricos completos!')

                                sql_insert = '''INSERT INTO info_paciente (Documento, Nombre, Edad, Sexo, EPS, Peso, Altura, Medida_cintura, Medida_cadera, Medida_cuello)
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

                                cursor.execute(
                                    sql_insert, (Id_, nombreap_, edad, Sexo, Eps, peso, altura, cintura, cadera, cuello))
                                cnx.commit()

                                print(
                                    'El paciente fue ingresado exitosamente a la base de datos.')
                                break

                            else:
                                print(
                                    '\nEl id ingresado ya existe en la base de datos, verifíquelo e intente de nuevo.')

                    elif menu1 == '2':
                        break

                    else:
                        print('Por favor ingrese una opción válida del menú.')

            if menu == '2':
                while True:
                    ID = input(
                        'Ingrese el documento de identidad del paciente a buscar: ')
                    ver1 = módulo.ver_paciente(ID)
                    cursor.execute(ver1)
                    ver1_ = cursor.fetchall()

                    if ver1_ == []:
                        print('El paciente no existe en la base de datos')
                        continue

                    else:
                        Genero = input(
                            'Digite si el paciente a buscar es 1. Hombre 2. Mujer:')

                        bring = módulo.traer_paciente(ID)
                        cursor.execute(bring)
                        bring_ = cursor.fetchall()
                        Imc = Módulos_de_tercero.IMC(
                            bring_[0][0], bring_[0][1])
                        print('IMC: ', Imc)
                        Ica = Módulos_de_tercero.ICA(
                            bring_[0][2], bring_[0][1])
                        print('ICA: ', Ica)
                        print('GEN: ', Genero)
                        if Genero == '1':
                            print(bring_)
                            GrasaH = Módulos_de_tercero.PorcentajeGrasaHombre(
                                bring_[0][2], bring_[0][4], bring_[0][1])
                            print('GRASA H: ', GrasaH)
                            if Imc < 18.5:
                                print('Menor 18.5')
                            if Imc >= 25:
                                if Ica <= 0.5 and 5 < GrasaH < 13:
                                    diagH1 = '''NO ADIPOSIDAD RIESGOSA, SE CLASIFICA COMO ATLETA por su porcentaje de grasa corporal 
                                        Su IMC INDICA QUE ES UNA PERSONA MUSCULAR'''
                                    print(f'\n{diagH1}')
                                elif Ica <= 0.5 and 14 < GrasaH < 17:
                                    diagH2 = '''NO ADIPOSIDAD RIESGOSA, SE CLASIFICA COMO fitness por su porcentaje de grasa corporal 
                                                Su IMC INDICA QUE ES UNA PERSONA MUSCULAR'''
                                    print(f'\n{diagH2}')
                                elif Imc < 30:
                                    diagH3 = 'El paciente se encuentra en sobrepeso'
                                    print(diagH3)
                                elif Imc >= 30:
                                    diagH4 = 'El paciente se encuentra en obecidad'
                                    print(diagH4)

                        elif Genero == '2':
                            diagnosticoGrasaM = Módulos_de_tercero.PorcentajeGrasaMujer(
                                bring_[0][2], bring_[0][3], bring_[0][4], bring_[0][1])

                        # Diagnóstico persona muscular

                    break

            elif menu == '3':
                while True:
                    menu3 = input('''
                    1. Agregar usuario
                    2. Cambiar contraseña 
                    3. Volver
                    >''')

                    if menu3 == '1':

                        nombre = input('Ingrese el nombre: ')
                        usuario = input('Ingresar usuario:')
                        contraseña = input('Contraseña: ')

                        dic_user = {'name': nombre,
                                    'user': usuario,
                                    'passwd': contraseña}

                        usuarios.append(dic_user)

                        file = open('admin/users.json',
                                    mode='w', encoding='utf8')
                        esc = json.dump(
                            usuarios, file, indent=4, ensure_ascii=False)
                        file.close()

                        print('El usuario fue agregado exitosamente')

                    elif menu3 == '2':
                        usuario_ = input(
                            '\nIngrese el usuario para el que desea cambiar la contraseña: ')

                        file = open('admin/users.json',
                                    mode='r', encoding='utf8')
                        lect = json.load(file)
                        file.close()

                        for i in lect:
                            if i['user'] == usuario_:
                                nuevacont_ = input(
                                    'Ingrese la nueva contraseña para el usuario ingresado: ')
                                i['passwd'] = nuevacont_
                                result = True
                                break
                            else:
                                result = False

                        if result == True:
                            archivo = open('admin/users.json',
                                           mode='w', encoding='utf8')
                            op = json.dump(
                                lect, archivo, indent=4, ensure_ascii=False)
                            archivo.close()

                            print('La contraseña fue actualizada exitosamente')

                        elif result == False:
                            print(
                                '\nEl nombre de usuario ingresado no existe, no es posible actualizar su contraseña.')
                            print(
                                'Verifique el nombre de usuario o agréguelo como usuario nuevo.')

                    elif menu3 == '3':
                        break
                    else:
                        print('¡¡ERROR!!')

            elif menu == '4':
                break

            else:
                print('¡¡ERROR!!')
        break

    else:
        print("\nEl nombre de usuario o la contraseña son incorrectos.")

--------------------------- SISTEMA DE DIAGNÓSTICO AUTOMÁTICO ---------------------------

Inicie sesión con sus credenciales.

¡Inicio de sesión exitoso!

Ingrese una opción del menú:
IMC:  22.67573696145125
ICA:  0.13095238095238096
GEN:  1
[(64.0, 1.68, 22.0, 20.0, 13.0)]
GRASA H:  -97.7358087548414

Ingrese una opción del menú:
